In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
train = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

In [1]:
#importing other modules
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

from sklearn import preprocessing 
#from sklearn.preprocessing import imputer
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import seaborn as sns
from sklearn.linear_model import LinearRegression 
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error,r2_score
%matplotlib inline
from sklearn import preprocessing 
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error,r2_score
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
from sklearn.datasets import make_regression
from sklearn.tree import DecisionTreeRegressor
from sklearn.datasets import make_regression
from sklearn import ensemble
from sklearn.ensemble import GradientBoostingRegressor

from matplotlib import pyplot

In [1]:
train.head()

In [1]:
# lets check missing data
NANs = pd.concat([train.isnull().sum(), test.isnull().sum()], axis=1, keys=['Train', 'Test'])
NANs[NANs.sum(axis=1) > 0].T

In [1]:
dfprep = pd.concat([train, test], keys=['train', 'test'])

In [1]:
dfprep.head()

In [1]:
# Missing values feature wise

sns.set_style("whitegrid")
missing = dfprep.isnull().sum()
missing = missing[missing > 0]
missing.sort_values(inplace=True)
missing.plot.bar()

In [1]:
dfprep.describe(include=["object"]).columns

In [1]:
num_col = dfprep.select_dtypes(include=np.number).columns
print("Numerical columns: \n",num_col)

cat_col = dfprep.select_dtypes(exclude=np.number).columns
print("Categorical columns: \n",cat_col)

In [1]:
_ = dfprep.hist(column=num_col, figsize = (18,18))

In [1]:
len(cat_col.to_list())

In [1]:
# Count plots of categorical variables

fig, axes = plt.subplots(4, 3, figsize=(16, 16))
plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.5, hspace=0.2)

for i, ax in enumerate(axes.ravel()):
    if i > 43:
        ax.set_visible(False)
        continue
    sns.countplot(y = cat_col[i], data=dfprep, ax=ax)

The VarianceThreshold from sklearn can be used for feature selection. It removes all zero-variance features that have the same value in all samples. It removes features whose variance doesn’t meet a particular threshold. Here we are using it for removing constant features

In [1]:
sel = VarianceThreshold(threshold=0)
sel.fit(dfprep[num_col])

In [1]:
sum(sel.get_support())

In [1]:
# now let's print the number of constant feautures
# (see how we use ~ to exclude non-constant features)

constant = dfprep[num_col].columns[~sel.get_support()]

len(constant)

In [1]:
#No constant Numerical features

In [1]:
#For checking constant Categorical features

constant_features = [
    feat for feat in dfprep[cat_col].columns if dfprep[feat].nunique() == 1
]

len(constant_features)

In [1]:
dfprep['Street'].value_counts(dropna=False)

In [1]:
#For checking constant Numerical features - again using std

constant_features = [
    feat for feat in dfprep[num_col].columns if dfprep[feat].std() == 1
]

len(constant_features)

In [1]:
# Dropping features that have many missing information or do not make much sense
# Features should not be dropped casually like this but still.. sometimes ;) 
dfprep.drop([ 'Heating', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1','BsmtQual', 'BsmtFinType2',
               'BsmtFullBath' ,'BsmtHalfBath', 'GarageYrBlt',  'GarageCond', 'WoodDeckSF',
               'OpenPorchSF', 'EnclosedPorch', 'LowQualFinSF', '3SsnPorch','GarageArea', 'ScreenPorch', 'PoolArea',  'Fence','PoolQC', 'MiscFeature','Utilities', 'RoofMatl', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'MiscVal','Alley'],
              axis=1, inplace=True)

In [1]:


#Filling NAs and converting dtypes

dfprep['MasVnrArea'] = dfprep['MasVnrArea'].fillna(dfprep['MasVnrArea'].mode()[0])

# MSSubClass as str
dfprep['Functional'] = dfprep['Functional'].astype(str)
dfprep['Functional'] = dfprep['Functional'].fillna(dfprep['Functional'].mode())


# LotFrontage  
dfprep['LotFrontage'] = dfprep['LotFrontage'].fillna(dfprep['LotFrontage'].mean())

# OverallCond 
dfprep.OverallCond = dfprep.OverallCond.astype(str)

# MasVnrType 
dfprep['MasVnrType'] = dfprep['MasVnrType'].fillna(dfprep['MasVnrType'].mode()[0])

# MSSubClass as str
dfprep['MSSubClass'] = dfprep['MSSubClass'].astype(str)

# MSZoning 
dfprep['MSZoning'] = dfprep['MSZoning'].fillna(dfprep['MSZoning'].mode()[0])


# TotalBsmtSF  
dfprep['TotalBsmtSF'] = dfprep['TotalBsmtSF'].fillna(0)


# KitchenAbvGr 
dfprep['KitchenAbvGr'] = dfprep['KitchenAbvGr'].astype(str)

# Electrical 
dfprep['Electrical'] = dfprep['Electrical'].fillna(dfprep['Electrical'].mode()[0])


# KitchenQual 
dfprep['KitchenQual'] = dfprep['KitchenQual'].fillna(dfprep['KitchenQual'].mode()[0])

# Yrsold and Mosold
dfprep['YrSold'] = dfprep['YrSold'].astype(str)
dfprep['MoSold'] = dfprep['MoSold'].astype(str)


# FireplaceQu  
dfprep['FireplaceQu'] = dfprep['FireplaceQu'].fillna('NoFP')

# GarageType, GarageFinish, GarageQual  
for col in ('GarageType', 'GarageFinish', 'GarageQual'):
    dfprep[col] = dfprep[col].fillna('No_GRGE')

# Combining SF all floors
dfprep['TotalSF'] = dfprep['TotalBsmtSF'] + dfprep['1stFlrSF'] + dfprep['2ndFlrSF']
dfprep.drop(['TotalBsmtSF', '1stFlrSF', '2ndFlrSF'], axis=1, inplace=True)    

# GarageCars 
dfprep['GarageCars'] = dfprep['GarageCars'].fillna(0.0)

# SaleType 
dfprep['SaleType'] = dfprep['SaleType'].fillna(dfprep['SaleType'].mode()[0])




In [1]:

# SalesPrice is skewed right ? 
ax = sns.distplot(dfprep['SalePrice'])

In [1]:
## Log transformation 
dfprep['SalePrice'] = np.log(dfprep['SalePrice'])
ax = sns.distplot(dfprep['SalePrice'])

In [1]:
#Other columns -- checking normalization for them
from scipy import stats
from scipy.stats import shapiro

num_col = dfprep.select_dtypes(include=np.number).columns



test_normality = lambda x: stats.shapiro(x.fillna(0))[1] < 0.01
normal = dfprep[num_col]
normal = normal.apply(test_normality)
print(not normal.any())

In [1]:
#### None of quantitative variables has normal distribution so ideally these should be transformed as well. But i will give it a pass

In [1]:
##Standardizing numeric data

#numeric_features = dfprep[num_col]
#numeric_features_standardized = (numeric_features - numeric_features.mean())/numeric_features.std()

##Scaling numeric data

#from sklearn import preprocessing
#dfprepp = dfprep[num_col].drop(['SalePrice'],axis=1)
#x = dfprepp.values #returns a numpy array
#min_max_scaler = preprocessing.MinMaxScaler()
#x_scaled = min_max_scaler.fit_transform(x)
#df_scaled = pd.DataFrame(x_scaled)
#df_scaled.columns = ['Id', 'LotFrontage', 'LotArea', 'OverallQual', 'YearBuilt','YearRemodAdd', 'GrLivArea', 'FullBath', 'HalfBath', 'BedroomAbvGr','TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 'TotalSF']
#dfprepp.update(df_scaled)
 
dfprepp = dfprep[num_col].drop(['SalePrice'],axis=1)
dfprepp_scaled = (dfprepp-dfprepp.min())/(dfprepp.max()-dfprepp.min())
dfprep.update(dfprepp_scaled)

In [1]:
dfprepp.head()

In [1]:
# Getting Dummies from all  categorical vars
for col in dfprep.dtypes[dfprep.dtypes == 'object'].index:
    for_dummy = dfprep.pop(col)
    dfprep = pd.concat([dfprep, pd.get_dummies(for_dummy )], axis=1)

In [1]:
# you just need the following 4 lines to rename duplicates
# dfprep is the dataframe that you want to rename duplicated columns

cols=pd.Series(dfprep.columns)

for dup in cols[cols.duplicated()].unique(): 
    cols[cols[cols == dup].index.values.tolist()] = [dup + '_' + str(i) if i != 0 else dup for i in range(sum(cols == dup))]

# rename the columns with the cols list.
dfprep.columns=cols


In [1]:
dfprep.head()

In [1]:
train_dfprep = dfprep.loc['train'].drop('Id', axis=1)
test_dfprep = dfprep.loc['test'].drop('Id', axis=1)

In [1]:
train_features = train_dfprep.drop(['SalePrice'], axis =1 )
train_label = train_dfprep['SalePrice']

In [1]:
# some fucntions for future model eval

def get_score(prediction, lables):    
    #print('R2: {}'.format(r2_score(prediction, lables)))
    print('RMSE: {}'.format(np.sqrt(mean_squared_error(prediction, lables))))

In [1]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

#mean_absolute_percentage_error(actual, prediction) --  gives you percent error

In [1]:
#### Feature selection using DecisionTreeRegressor

In [1]:
pd.options.display.float_format = '{:,.2f}'.format
# define dataset
train_dfprep = dfprep.loc['train']
test_dfprep = dfprep.loc['test']
train_features = train_dfprep.drop(['SalePrice'], axis =1 )
train_label = train_dfprep['SalePrice']
x_train, x_test, y_train, y_test = train_test_split(train_features, train_label, test_size=0.1, random_state=200)
#X, y = make_regression(n_samples=1000, n_features=10, n_informative=5, random_state=1)

# define the model
model = DecisionTreeRegressor()
# fit the model
model.fit(x_train, y_train)
# get importance
importance = model.feature_importances_
# summarize feature importance
name = x_train.columns.to_list()
important_features = pd.DataFrame(list(zip(name, importance)), columns = ['Name','Importance']).sort_values(by = 'Importance', ascending = False)
print(important_features)


In [1]:
important_features.head(15)['Name'].to_list()

In [1]:
dfprep2 = dfprep.copy()
dfprep2 = dfprep2[['OverallQual',
 'TotalSF',
 'Y',
 'LotArea',
 'GarageCars',
 'YearRemodAdd',
 'GrLivArea',
 'YearBuilt',
 'Detchd',
 'C (all)',
 'TA_5',
 'Id',
 'NoFP',
 'LotFrontage',
 'Brk Cmn','SalePrice']]

train_dfprep2 = dfprep2.loc['train']
test_dfprep2 = dfprep2.loc['test']
train_features = train_dfprep2.drop(['SalePrice'], axis =1 )
train_label = train_dfprep2['SalePrice']
x_train, x_test, y_train, y_test = train_test_split(train_features, train_label, test_size=0.1, random_state=200)

GBest2 = ensemble.GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05, max_depth=3, max_features='sqrt',
                                               min_samples_leaf=15, min_samples_split=10, loss='huber').fit(x_train, y_train)

GBest2.score(x_test, y_test)

In [1]:
### Feature selection using RFE and RFE with CV. Lets go..

In [1]:
train_dfprep2 = dfprep2.loc['train']
test_dfprep2 = dfprep2.loc['test']
train_features = train_dfprep2.drop(['SalePrice'], axis =1 )
train_label = train_dfprep2['SalePrice']
x_train, x_test, y_train, y_test = train_test_split(train_features, train_label, test_size=0.1, random_state=200)


lm = LinearRegression()
lm.fit(x_train, y_train)

rfe = RFE(lm, n_features_to_select = 15)   
rfe = RFE(lm)   
rfe = rfe.fit(x_train, y_train)

selected_rfe_features = pd.DataFrame({'Feature':list(x_train.columns),
                                      'Ranking':rfe.ranking_})
selected_rfe_features.sort_values(by='Ranking')

x_train = rfe.transform(x_train)
x_test = rfe.transform(x_test)


lm2 = LinearRegression()
rfe_model = lm2.fit(x_train, y_train)

selected_rfe_features

In [1]:
actual     = y_test
prediction = rfe_model.predict(x_test)

In [1]:
r2_score(actual, prediction)

In [1]:
### Ah.. a lil bad than previous one. Lets try with CV now.

In [1]:
#RFE with CV

train_dfprep2 = dfprep2.loc['train']
test_dfprep2 = dfprep2.loc['test']
train_features = train_dfprep2.drop(['SalePrice'], axis =1 )
train_label = train_dfprep2['SalePrice']
x_train, x_test, y_train, y_test = train_test_split(train_features, train_label, test_size=0.1, random_state=200)


rfecv = LinearRegression()
rfecv.fit(x_train, y_train)


rfecv = RFECV(estimator=rfecv, step=1, cv=5, scoring='r2')
rfecv = rfecv.fit(x_train, y_train)
print('Optimal number of features :', rfecv.n_features_)
print('Best features :', x_train.columns[rfecv.support_])

In [1]:
rfecv.grid_scores_

In [1]:
#search_grid={'n_estimators':[2000,3000],'learning_rate':[.001,0.01,.1],'max_depth':[1,2,6],'subsample':[.5,.75,1],'random_state':[1]}
#search=GridSearchCV(estimator=GBR,param_grid=search_grid,n_jobs=1,cv=5)
#search.fit(X,y)
#search.best_params_
#search.best_score_

In [1]:
x_train_f = rfecv.transform(x_train)
x_test_f = rfecv.transform(x_test)

GBest3 = ensemble.GradientBoostingRegressor(n_estimators=3000, learning_rate=0.06, max_depth=3, max_features='sqrt',
                                               min_samples_leaf=15, min_samples_split=10, loss='huber').fit(x_train_f, y_train)

GBest3.score(x_test_f, y_test)

In [1]:
#Hmm..not that bad !!

In [1]:
actual     = y_test
prediction = GBest3.predict(x_test_f)

In [1]:
r2_score(actual, prediction) # basically the same thing as above..

In [1]:
# Feature selection using DecisionTreeRegressor has given us the good features till now so lets use it for final prediction

In [1]:
# on our created testing dataset

GBest2.score(x_test, y_test)

In [1]:
mean_squared_error(prediction, actual)

In [1]:
testdata = dfprep.loc['test']

In [1]:
# Using features chosen by reg tree

testdata = testdata[['OverallQual',
 'TotalSF',
 'Y',
 'LotArea',
 'GarageCars',
 'YearRemodAdd',
 'GrLivArea',
 'YearBuilt',
 'Detchd',
 'C (all)',
 'TA_5',
 'Id',
 'NoFP',
 'LotFrontage',
 'Brk Cmn']]

In [1]:
SalePrice = pd.Series(np.exp(GBest2.predict(testdata)))

In [1]:
test = test['Id']

In [1]:
output_df = pd.DataFrame({'Id':test,'SalePrice':SalePrice})

In [1]:
output_df.head()

That was the prediction of final test result.
There are tons of things that can be improved. 
Do let me know your feedbacks. Thanks to stackoverflow and top kagglers in this learning competition from where i got good ideas to implement this one. See ya !